# Hyperparameter Optimization All models

## Rationale:
In order to make a fair comparisson in performance, we are to tune the hyperparamters for all the models so that we can optimize the performance of each model and then compared then to see if we may be affecting because a bad initial setup.

## Methodology:
We conducted hyperparameter optimization using the Optuna library, which provides a powerful and efficient platform for automated hyperparameter tuning.

We are to Use the ```LightGBM Integration``` which optimizes the hyperparamters in a sequential way and focuses on the parameters that have the highest impact on model performance for Boosting models:



* **lambda_l1**
* **lambda_l2**
* **num_leaves**
* **feature_fraction**
* **bagging_fraction**
* **bagging_freq**
* **min_child_samples**

For the **num_estimators** and **learning_rate** we are to use the ones found on the best model optimization.

## Conclusions:
The results of our hyperparameter optimization study yielded valuable insights into improving the LightGBM model's performance. For all the models we achived an increased in performance as compared with the initial model comparisson. (see ```feature-selection``` notebook) Yet, the boruta model continues to have the highest performance.


| Model             | out_of_fold | validation |
|-------------------|-------------|------------|
| boruta [426]      | 0.795680    | 0.800874   |
| ensemble [820]    | 0.792880    | 0.799627   |
| fw [329]          | 0.792839    | 0.799425   |
| all features [1147]| 0.791760    | 0.799144   |
| optuna [555]      | 0.788259    | 0.795938   |




In [1]:
%load_ext autoreload
%autoreload 2

import cloudpickle as cp

import numpy as np
import pandas as pd

import lightgbm  as lgbm
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from copy import deepcopy

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score

import warnings;warnings.filterwarnings("ignore")

from optuna import visualization as optunaviz

import optuna
import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation
from optuna.logging import set_verbosity

set_verbosity(optuna.logging.ERROR)

import sys
sys.path.append("../")

# local imports
from src.learner_params import target_column, space_column, boruta_learner_params, test_params
from utils.functions__utils import find_constraint

from utils.feature_selection_lists import fw_features, boruta_features, optuna_features, ensemble_features
from utils.features_lists import all_features_list

from src.learner_params import MODEL_PARAMS

from utils.functions__training import model_pipeline

from src.learner_params import params_all, params_ensemble, params_fw, params_optuna

In [2]:
train_df = pd.read_pickle("../data/train_df.pkl")
validation_df = pd.read_pickle("../data/validation_df.pkl")

train, test = train_test_split(train_df, random_state=42, test_size=.2)

### All features

In [9]:
%%time

dtrain = lgbm.Dataset(train[all_features_list], label=train[target_column])
dval = lgb.Dataset(test[all_features_list], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[all_features_list],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

all_features_best_params = bst.params
print("Best params:", all_features_best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.788152:  14%|7    | 1/7 [05:57<35:46, 357.76s/it]

Early stopping, best iteration is:
[3053]	valid_0's auc: 0.893864	valid_1's auc: 0.788152


feature_fraction, val_score: 0.788152:  14%|7    | 1/7 [05:57<35:46, 357.76s/it]

...

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2178]	valid_0's auc: 0.962575	valid_1's auc: 0.791803


min_data_in_leaf, val_score: 0.792268:  80%|#### | 4/5 [36:50<09:12, 552.45s/it]

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2198]	valid_0's auc: 0.963067	valid_1's auc: 0.792039


min_data_in_leaf, val_score: 0.792268: 100%|#####| 5/5 [46:09<00:00, 553.81s/it]


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 1.0073717100534047e-08, 'lambda_l2': 8.932762924966607, 'num_leaves': 150, 'feature_fraction': 0.5, 'bagging_fraction': 0.6580775817581002, 'bagging_freq': 7, 'min_child_samples': 20, 'num_iterations': 5926}
CPU times: user 1d 21h 31min 39s, sys: 2h 19min 34s, total: 1d 23h 51min 14s
Wall time: 9h 10min 38s


In [14]:
all_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 1.0073717100534047e-08,
 'lambda_l2': 8.932762924966607,
 'num_leaves': 150,
 'feature_fraction': 0.5,
 'bagging_fraction': 0.6580775817581002,
 'bagging_freq': 7,
 'min_child_samples': 20,
 'num_iterations': 5926}

### MrMr Features

In [10]:
%%time
dtrain = lgbm.Dataset(train[fw_features], label=train[target_column])
dval = lgb.Dataset(test[fw_features], label=test[target_column])
params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[fw_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

fw_features_best_params = bst.params
print("Best params:", fw_features_best_params)



feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.787981:  14%|7    | 1/7 [03:07<18:42, 187.07s/it]

Early stopping, best iteration is:
[3643]	valid_0's auc: 0.902415	valid_1's auc: 0.787981
Training until validation scores don't improve for 150 rounds


feature_fraction, val_score: 0.787981:  29%|#4   | 2/7 [05:31<13:29, 161.81s/it]

...

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2447]	valid_0's auc: 0.942127	valid_1's auc: 0.791391


min_data_in_leaf, val_score: 0.791447:  80%|#### | 4/5 [15:11<03:45, 225.98s/it]

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2547]	valid_0's auc: 0.945589	valid_1's auc: 0.791049


min_data_in_leaf, val_score: 0.791447: 100%|#####| 5/5 [19:17<00:00, 231.48s/it]


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 0.17381521305946443, 'lambda_l2': 8.821433034912198, 'num_leaves': 111, 'feature_fraction': 0.4, 'bagging_fraction': 0.7396714409529886, 'bagging_freq': 6, 'min_child_samples': 20, 'num_iterations': 5926}
CPU times: user 16h 2min 24s, sys: 1h 15min 50s, total: 17h 18min 15s
Wall time: 3h 37min 36s


In [13]:
fw_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 0.17381521305946443,
 'lambda_l2': 8.821433034912198,
 'num_leaves': 111,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.7396714409529886,
 'bagging_freq': 6,
 'min_child_samples': 20,
 'num_iterations': 5926}

### Ensemble features

In [15]:
%%time
dtrain = lgbm.Dataset(train[ensemble_features], label=train[target_column])
dval = lgb.Dataset(test[ensemble_features], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[ensemble_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

ensemble_features_best_params = bst.params
print("Best params:", ensemble_features_best_params)





  0%|                                                     | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds




feature_fraction, val_score: 0.785942:   0%|              | 0/7 [03:53<?, ?it/s]

feature_fraction, val_score: 0.785942:  14%|7    | 1/7 [03:53<23:19, 233.17s/it]

feature_fraction, val_score: 0.785942:  14%|7    | 1/7 [03:53<23:19, 233.17s/it]

Early stopping, best iteration is:
[2283]	valid_0's auc: 0.875706	valid_1's auc: 0.785942
Training until validation scores don't improve for 150 rounds




feature_fraction, val_score: 0.786267:  14%|7    | 1/7 [08:00<23:19, 233.17s/it]

feature_fraction, val_score: 0.786267:  29%|#4   | 2/7 [08:00<20:06, 241.38s/it]

feature_fraction, val_score: 0.786267:  29%|#4   | 2/7 [08:00<20:06, 241.38s/it]

...

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2549]	valid_0's auc: 0.958641	valid_1's auc: 0.79227




min_data_in_leaf, val_score: 0.792392:  60%|###  | 3/5 [33:05<16:26, 493.19s/it]

min_data_in_leaf, val_score: 0.792392:  80%|#### | 4/5 [33:05<08:24, 504.57s/it]

min_data_in_leaf, val_score: 0.792392:  80%|#### | 4/5 [33:05<08:24, 504.57s/it]

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2410]	valid_0's auc: 0.954306	valid_1's auc: 0.792254




min_data_in_leaf, val_score: 0.792392:  80%|#### | 4/5 [41:18<08:24, 504.57s/it]

min_data_in_leaf, val_score: 0.792392: 100%|#####| 5/5 [41:19<00:00, 495.81s/it]


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 3.778715480993499e-08, 'lambda_l2': 9.701943176300759, 'num_leaves': 124, 'feature_fraction': 0.41600000000000004, 'bagging_fraction': 0.7401877344450646, 'bagging_freq': 2, 'min_child_samples': 100, 'num_iterations': 5926}
CPU times: user 1d 8h 29min 24s, sys: 1h 47min 29s, total: 1d 10h 16min 53s
Wall time: 6h 46min 1s


In [16]:
ensemble_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 3.778715480993499e-08,
 'lambda_l2': 9.701943176300759,
 'num_leaves': 124,
 'feature_fraction': 0.41600000000000004,
 'bagging_fraction': 0.7401877344450646,
 'bagging_freq': 2,
 'min_child_samples': 100,
 'num_iterations': 5926}

### Optuna Features

In [17]:
%%time
dtrain = lgbm.Dataset(train[optuna_features], label=train[target_column])
dval = lgb.Dataset(test[optuna_features], label=test[target_column])

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "n_estimators":5926,
    "learning_rate": 0.005603627873630697
}

bst = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dval],
    callbacks=[early_stopping(150)],
    show_progress_bar = True
)

preds = bst.predict(test[optuna_features],num_iteration=bst.best_iteration)
score = roc_auc_score(test[target_column], preds)

optuna_features_best_params = bst.params
print("Best params:", optuna_features_best_params)





  0%|                                                     | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: -inf:   0%|                  | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 150 rounds




feature_fraction, val_score: 0.784149:   0%|              | 0/7 [02:43<?, ?it/s]

feature_fraction, val_score: 0.784149:  14%|7    | 1/7 [02:43<16:18, 163.04s/it]

feature_fraction, val_score: 0.784149:  14%|7    | 1/7 [02:43<16:18, 163.04s/it]

Early stopping, best iteration is:
[2285]	valid_0's auc: 0.867886	valid_1's auc: 0.784149
Training until validation scores don't improve for 150 rounds




feature_fraction, val_score: 0.785450:  14%|7    | 1/7 [06:38<16:18, 163.04s/it]

feature_fraction, val_score: 0.785450:  29%|#4   | 2/7 [06:38<17:09, 205.85s/it]

...

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1660]	valid_0's auc: 0.965574	valid_1's auc: 0.788016




min_data_in_leaf, val_score: 0.788016:  60%|###  | 3/5 [18:06<09:11, 275.62s/it]

min_data_in_leaf, val_score: 0.788016:  80%|#### | 4/5 [18:06<04:29, 269.81s/it]

min_data_in_leaf, val_score: 0.788016:  80%|#### | 4/5 [18:06<04:29, 269.81s/it]

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1559]	valid_0's auc: 0.961629	valid_1's auc: 0.787445




min_data_in_leaf, val_score: 0.788016:  80%|#### | 4/5 [22:17<04:29, 269.81s/it]

min_data_in_leaf, val_score: 0.788016: 100%|#####| 5/5 [22:17<00:00, 267.43s/it]


Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.005603627873630697, 'feature_pre_filter': False, 'lambda_l1': 3.201016964067897e-07, 'lambda_l2': 4.0276123590778266, 'num_leaves': 200, 'feature_fraction': 0.48000000000000004, 'bagging_fraction': 0.8100007780166677, 'bagging_freq': 3, 'min_child_samples': 100, 'num_iterations': 5926}
CPU times: user 17h 26min 4s, sys: 1h 42min 10s, total: 19h 8min 15s
Wall time: 4h 17min 13s


In [18]:
optuna_features_best_params

{'objective': 'binary',
 'metric': 'auc',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.005603627873630697,
 'feature_pre_filter': False,
 'lambda_l1': 3.201016964067897e-07,
 'lambda_l2': 4.0276123590778266,
 'num_leaves': 200,
 'feature_fraction': 0.48000000000000004,
 'bagging_fraction': 0.8100007780166677,
 'bagging_freq': 3,
 'min_child_samples': 100,
 'num_iterations': 5926}

### Model training

In [3]:
save_estimator_path = "../model_files/fw_learner.pkl"
fw_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = params_fw,
                            target_column = target_column,
                            features = fw_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )
with open(save_estimator_path, "wb") as context:
    cp.dump(fw_logs["lgbm_classification_learner"], context)

2023-09-27T17:10:14 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-27T17:10:14 | INFO | Training for fold 1
2023-09-27T17:18:11 | INFO | Training for fold 2
2023-09-27T17:26:08 | INFO | Training for fold 3
2023-09-27T17:34:12 | INFO | CV training finished!
2023-09-27T17:34:12 | INFO | Training the model in the full dataset...
2023-09-27T17:43:44 | INFO | Training process finished!
2023-09-27T17:43:44 | INFO | Calculating metrics...
2023-09-27T17:43:44 | INFO | Full process finished in 33.52 minutes.
2023-09-27T17:43:44 | INFO | Saving the predict function.
2023-09-27T17:43:44 | INFO | Predict function saved.


In [4]:
fw_logs["data"]["oof_df"].to_pickle("../data/fw_oof_df.pkl")

In [5]:
save_estimator_path = "../model_files/optuna_learner.pkl"
optuna_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = params_optuna,
                            target_column = target_column,
                            features = optuna_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )
with open(save_estimator_path, "wb") as context:
    cp.dump(optuna_logs["lgbm_classification_learner"], context)

2023-09-27T17:43:49 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-27T17:43:49 | INFO | Training for fold 1
2023-09-27T17:56:19 | INFO | Training for fold 2
2023-09-27T18:08:53 | INFO | Training for fold 3
2023-09-27T18:21:37 | INFO | CV training finished!
2023-09-27T18:21:37 | INFO | Training the model in the full dataset...
2023-09-27T18:36:31 | INFO | Training process finished!
2023-09-27T18:36:31 | INFO | Calculating metrics...
2023-09-27T18:36:31 | INFO | Full process finished in 52.75 minutes.
2023-09-27T18:36:31 | INFO | Saving the predict function.
2023-09-27T18:36:31 | INFO | Predict function saved.


In [6]:
optuna_logs["data"]["oof_df"].to_pickle("../data/optuna_oof_df.pkl")

In [7]:
save_estimator_path = "../model_files/ensemble_learner.pkl"
ensemble_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = test_params,
                            target_column = target_column,
                            features = ensemble_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )
with open(save_estimator_path, "wb") as context:
    cp.dump(ensemble_logs["lgbm_classification_learner"], context)

2023-09-27T18:36:38 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-27T18:36:39 | INFO | Training for fold 1
2023-09-27T18:40:12 | INFO | Training for fold 2
2023-09-27T18:43:54 | INFO | Training for fold 3
2023-09-27T18:47:45 | INFO | CV training finished!
2023-09-27T18:47:45 | INFO | Training the model in the full dataset...
2023-09-27T18:52:51 | INFO | Training process finished!
2023-09-27T18:52:51 | INFO | Calculating metrics...
2023-09-27T18:52:52 | INFO | Full process finished in 16.26 minutes.
2023-09-27T18:52:52 | INFO | Saving the predict function.
2023-09-27T18:52:52 | INFO | Predict function saved.


In [8]:
ensemble_logs["data"]["oof_df"].to_pickle("../data/ensemble_oof_df.pkl")

In [9]:
save_estimator_path = "../model_files/all_learner.pkl"
all_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = params_all,
                            target_column = target_column,
                            features = all_features_list,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )
with open(save_estimator_path, "wb") as context:
    cp.dump(all_logs["lgbm_classification_learner"], context)

2023-09-27T18:52:56 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-27T18:52:57 | INFO | Training for fold 1
2023-09-27T19:14:48 | INFO | Training for fold 2
2023-09-27T19:36:57 | INFO | Training for fold 3
2023-09-27T20:00:35 | INFO | CV training finished!
2023-09-27T20:00:35 | INFO | Training the model in the full dataset...
2023-09-27T20:29:24 | INFO | Training process finished!
2023-09-27T20:29:24 | INFO | Calculating metrics...
2023-09-27T20:29:24 | INFO | Full process finished in 96.49 minutes.
2023-09-27T20:29:24 | INFO | Saving the predict function.
2023-09-27T20:29:25 | INFO | Predict function saved.


In [10]:
all_logs["data"]["oof_df"].to_pickle("../data/all_oof_df.pkl")

In [11]:
save_estimator_path = "../model_files/boruta_learner.pkl"
boruta_logs = model_pipeline(train_df = train_df,
                            validation_df = validation_df,
                            params = MODEL_PARAMS,
                            target_column = target_column,
                            features = boruta_features,
                            cv = 3,
                            random_state = 42,
                            apply_shap = False,
                            save_estimator_path=None
                          )
with open(save_estimator_path, "wb") as context:
    cp.dump(boruta_logs["lgbm_classification_learner"], context)

2023-09-27T20:29:30 | INFO | Starting pipeline: Generating 3 k-fold training...
2023-09-27T20:29:30 | INFO | Training for fold 1
2023-09-27T20:33:38 | INFO | Training for fold 2
2023-09-27T20:37:34 | INFO | Training for fold 3
2023-09-27T20:41:32 | INFO | CV training finished!
2023-09-27T20:41:32 | INFO | Training the model in the full dataset...
2023-09-27T20:46:36 | INFO | Training process finished!
2023-09-27T20:46:36 | INFO | Calculating metrics...
2023-09-27T20:46:36 | INFO | Full process finished in 17.12 minutes.
2023-09-27T20:46:36 | INFO | Saving the predict function.
2023-09-27T20:46:36 | INFO | Predict function saved.


In [12]:
boruta_logs["data"]["oof_df"].to_pickle("../data/boruta_oof_df.pkl")

### Model evaluation

In [17]:
model_metrics  ={}
models = [all_logs, fw_logs, boruta_logs, optuna_logs, ensemble_logs]
names = ["all features", "fw", "boruta", "optuna", "ensemble"]
sizes = [len(all_features_list), len(fw_features), len(boruta_features), len(optuna_features), len(ensemble_features)]

for model, name, size in zip(models, names, sizes):
    model_metrics[f"{name} [{size}]"] = model["metrics"]["roc_auc"]
pd.DataFrame(model_metrics).T.sort_values(by = "validation", ascending = False)

,out_of_fold,validation
boruta [426],0.795680,0.800874
ensemble [820],0.792880,0.799627
fw [329],0.792839,0.799425
all features [1147],0.791760,0.799144
optuna [555],0.788259,0.795938
